In [0]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import lightgbm as lgb

from components.data_prep import *
from components.preprocess import *
from model.nn import *
from components.helper import *
from components.config import *
#from ingestion import *
import mlflow
cdp_path='/dbfs/mnt/cdpprod/Customer_Profile_Aggregates/'
mgm_reward = spark.read.parquet('dbfs:/mnt/proddatalake/prod/CFA/CFA_overall_lvl')
rc = spark.read.format("delta").load('dbfs:/mnt/edhprodenrich/Enterprise Data Store/Secured/data/Cleanse/RCX/RCX_enums/CurrentState')
la = spark.read.format("delta").load("dbfs:/mnt/edhprodenrich/Enterprise Data Store/Secured/data/Cleanse/RCX/RCX_loyalty_activities/CurrentState")

In [0]:
trip_data.select('property_name').distinct().display()

In [0]:
cust_profile_path = 'dbfs:/mnt/cdpprod/Customer_Profile_Customer_nopii/'+'2025-02-03'+'/';
customer_data = spark.read.parquet(cust_profile_path)
cfa = spark.read.parquet("dbfs:/mnt/proddatalake/dev/CFA/CFA_overall_lvl_Region_Beau")
trip_data = spark.read.parquet('dbfs:/mnt/cdpprod/Customer_Profile_Aggregates/2025-02-05')
trip_data = trip_data.groupBy('guest_id','TripStart','TripEnd').agg(F.sum('net_gaming_revenue').alias('win_loss'))

In [0]:
regional = "Beau Rivage"

In [0]:
yesterday=str(max(os.listdir(cdp_path)))

trip_data = spark.read.parquet('dbfs:/mnt/cdpprod/Customer_Profile_Aggregates/'+yesterday+'/')\
            .withColumn('Site', F.when(F.col('SiteGroup') == 'LAS', 'Vegas').otherwise('Region'))\
            .filter(F.col('Mnth')>='2016-01-01')\
            .filter((F.col('TripRvMgmt_Segment')!='Convention')|F.col('TripRvMgmt_Segment').isNull())
trip_data = trip_data.withColumn("TripID",F.concat(F.col("Guest_ID"),F.lit('_'), F.col("TripStart"),F.lit('_'),F.col("TripEnd")))

CPA = trip_data.where("property_name != 'BetMGM' and tripstart < '2024-12-31'").select('guest_id','Property_Name','Department','TripStart','TripEnd','TripStartMlifeTier', 'TripGamingDays','TripID')
trip_CPA = CPA.groupBy('guest_id','TripStart','TripEnd', 'TripID').agg(F.count('Department').alias('dept_num'), F.max('Property_Name').alias('property_name'),
                                                                              F.max('TripGamingDays').alias('TripGamingDays'), F.max('TripStartMlifeTier').alias('TripStartMlifeTier'))
trip_br = trip_CPA.where(F.col('property_name').contains(regional))
trip_br_2022 = trip_br.where('TripStart between "2022-01-01" and "2022-12-31"')
trip_br_2023 = trip_br.where('TripStart between "2023-01-01" and "2023-12-31"')
trip_br_2024 = trip_br.where('TripStart between "2024-01-01" and "2024-12-31"')

trip_spec = trip_br_2024.join(trip_br_2023, on='guest_id', how='inner').select('guest_id').distinct()

In [0]:
# Tier Credits Earning History from RCX
tc_2022 = spark.read.parquet(f'dbfs:/mnt/proddatalake/dev/RCX/TC_2022.parquet')
tc_2023 = spark.read.parquet(f'dbfs:/mnt/proddatalake/dev/RCX/TC_2023.parquet')
tc_2024 = spark.read.parquet(f'dbfs:/mnt/proddatalake/dev/RCX/TC_2024.parquet')
#tc_combined = tc_2022.union(tc_2023).union(tc_2024)
# 2023 only
tc_combined = tc_2023.union(tc_2024)

tc_combined = tc_combined.groupBy('playerid').agg(
    F.sum('tiercredit').alias('total_tc'), 
    F.sum(F.when(F.col('site_name').contains(regional), F.col('tiercredit')).otherwise(F.lit(0))).alias('regional_tc')
)

tc_combined = tc_combined.join(mgm_reward, F.col('playerid') == F.col('MLifeID'), how='inner').select(*tc_combined.columns, 'guest_id')

# Filtering Borgata Dominant Players
dominant_players = tc_combined.where('regional_tc > (total_tc * 0.8)').select('guest_id')
# Filtering Trip Data
trip = trip_CPA.join(trip_spec, on='guest_id', how='inner').join(dominant_players, on='guest_id', how='inner')

In [0]:
TC_trip_2023 = TC_trip_formulation(2023, trip, spark)
TC_customer_hist_2023 = TC_trip_2023.groupby('guest_id','calendar_year','change_assigned').agg(
    F.count('*').alias('trip_num'),
    F.min('tripstart').alias('earliest_trip'),
    F.sum(F.date_diff('tripend', 'tripstart')+1).alias('trip_days'),
    F.sum('TotalTierCredit').alias('TotalTierCredit'),
    F.sum('gaming_tc').alias('gaming_tc'),
    F.sum('non_gaming_tc').alias('non_gaming_tc'),
    F.sum(F.when(F.col('TripLodgingStatus') == "01 - In House", 1).otherwise(0)).alias('lodger_trip'),
    F.sum(F.when(F.col('TripLodgingStatus') == "02 - Local", 1).otherwise(0)).alias('local_trip'),
    F.sum(F.when(F.col('TripLodgingStatus') == "03 - Drop In", 1).otherwise(0)).alias('non_loager_trip'),
    F.min('tier').alias('tier'),
    F.max('tier_before').alias('pro_tier'),
    F.max('mt_reason').alias('reason_code'),
    F.max('mt_subreason').alias('subreason_code'),
    F.max('mth_reason').alias('mth_reason_code'),
    F.max('mth_subreason').alias('mth_subreason_code')
).withColumn(
    'earliest_trip', F.date_diff(F.lit('2023-12-31'),'earliest_trip')
).withColumn(
    'trip_tier',
    F.coalesce('pro_tier', F.substring("tier", 2, 2).cast('int'))
).withColumn(
    'target_TC', 
    F.when(F.col('trip_tier')==1, 20000)
    .when(F.col('trip_tier')==2, 75000)
    .when(F.col('trip_tier')==3, 200000)
    .otherwise(-1)
).filter('target_tc != -1')

In [0]:
hist_combined_2023, hist_special_promotion_2023 = success_unsuccess_combination(TC_customer_hist_2023, rc, mgm_reward,2023, 0.8, spark)

In [0]:
TC_trip_2024 = TC_trip_formulation_daily_model(2024, trip, spark)
train_2024 = TC_trip_2024.groupby('guest_id','calendar_year','change_assigned').agg(
    F.count('*').alias('trip_num'), 
    F.sum('TotalTierCredit').alias('TotalTierCredit'),
    F.sum('gaming_tc').alias('gaming_tc'),
    F.sum('non_gaming_tc').alias('non_gaming_tc'),
    F.min('tier').alias('tier'),
    F.max('tier_before').alias('pro_tier'),
    F.max('mt_reason').alias('reason_code'),
    F.max('mt_subreason').alias('subreason_code'),
    F.max('mth_reason').alias('mth_reason_code'),
    F.max('mth_subreason').alias('mth_subreason_code'),
    F.max('tier_after').alias('tier_after')
).withColumn(
    'trip_tier',
    F.coalesce('pro_tier', F.substring("tier", 2, 2).cast('int'))
).withColumn(
    'target_TC', 
    F.when(F.col('trip_tier')==1, 20000)
    .when(F.col('trip_tier')==2, 75000)
    .when(F.col('trip_tier')==3, 200000)
    .otherwise(-1)
).filter('target_tc != -1')

windowSpec = Window.partitionBy("guest_id").orderBy("TripStart")
TC_trip_2024 = TC_trip_2024.withColumn("row_number", F.row_number().over(windowSpec))


In [0]:
year = 2024
TC = spark.read.parquet(f'dbfs:/mnt/proddatalake/dev/RCX/TC_{year}.parquet')
#train = get_train_daily(TC, hist, hist_special_promotion, 2024, TC_trip_2024, train_2024, mgm_reward, rc, la)
# 2023 only
train = get_train_daily(TC, hist_combined_2023, hist_special_promotion_2023, 2024, TC_trip_2024, train_2024, mgm_reward, rc, la)
error= train.where('cuml_tc > target_tc and (transactiondate != change_assigned or change_assigned is null)').select('train_guest_id').distinct()
train = train.join(error, on = 'train_guest_id', how='leftanti')

In [0]:
train.select(*FEATURE_NAMES_RETURN).describe().display()

In [0]:
# mlflow.set_registry_uri("databricks-uc")
# scaler_model_path = "models:/data_science_mart.tierimminent_cleaned.minmax_scaler_borgata_return@active"
# features = ['tiercredit', 'remaining_days', 'days_num', 'cuml_tc', 'trip_duration','trip_day','TripLodgingStatus', 'highest_trip_tier', 'gtc_percentage', 'ngtc_percentage', 'total_tc', 'trip_num', 'trip_days', 'earliest_trip', 'lodger_percentage', 'local_percentage', 'target_tc']
# train = apply_minmax(train, features, scaler_model_path)
# train.write.mode("overwrite").parquet('/mnt/proddatalake/dev/TierImminent/data/BR_test.parquet')
t = spark.read.parquet('dbfs:/mnt/proddatalake/dev/TierImminent/data/BR_test.parquet')

# unsucess_id = t.groupby('train_guest_id').agg(F.count('change_assigned').alias('count')).where('count = 0').sample(withReplacement=False, fraction=0.5, seed=269)
# train_final = t.join(unsucess_id, on = 'train_guest_id', how='leftanti').select(*train.columns)
train_final = t.where('(change_assigned != transactiondate) or change_assigned is null')

In [0]:
loaded_model = mlflow.pytorch.load_model(f"models:/data_science_mart.tierimminent_cleaned.borgata_lstm_model@newtrained", map_location=torch.device('cpu'))

In [0]:
from tqdm import tqdm
# outputs_ls, labels_ls, y_pred_ls_2, y_pred_ls_5, y_pred_ls_35 = [], [], [], [], []
outputs_ls = []
labels_ls = []
remaining_ls = []
new_outputs_ls = []
id_ls = []
X, y, l = None, np.array([]), np.array([])
tier = pd.DataFrame(columns=['id','remaining_dyas','tier'])
features = ['tiercredit', 'remaining_days', 'days_num', 'cuml_tc', 'trip_duration','trip_day','TripLodgingStatus', 'highest_trip_tier', 'gtc_percentage', 'ngtc_percentage', 'total_tc', 'trip_num', 'trip_days', 'earliest_trip', 'lodger_percentage', 'local_percentage', 'target_tc']
#features = ['tiercredit', 'remaining_days', 'days_num', 'cuml_tc', 'trip_duration','trip_day','TripLodgingStatus', 'lodger_percentage', 'local_percentage', 'target_tc']
for j in range(len(features)):
    features[j] = features[j]+'_scaled'
for i in tqdm(range(365, 0, -1)):
    cus_list = train_final.filter(train_final.remaining_days == i).select(F.col('train_guest_id').alias('id')).distinct()
    temp_df = train_final.join(cus_list, (train_final.train_guest_id == cus_list.id) & (train_final.remaining_days >= i), how = 'inner')
    temp_df = temp_df.withColumn('rn', F.row_number().over(Window.partitionBy('train_guest_id').orderBy('remaining_days'))).where('rn <= 6')
    # t = temp_df.groupby('train_guest_id','remaining_days').agg(F.min('target_tc').alias('tier')).where(f'remaining_days == {i}')
    # t = t.toPandas()
    # tier = pd.concat([tier, t], ignore_index=True)
 
    train_np, label, length, train_id = create_sequences_and_labels(temp_df, features, 6)
    if X is None:
        X = train_np
    else:
        X = np.append(X, train_np, axis = 0)
    y = np.append(y, label)
    l = np.append(l, length)
    
    # X = torch.tensor(train_np, dtype=torch.float32).cpu()
    # len_X = torch.tensor(length, dtype=torch.float32).cpu()
    # with torch.no_grad():
    #     outputs = loaded_model(X, len_X).squeeze()
    # outputs = torch.sigmoid(outputs)

    test_X = torch.tensor(train_np, dtype=torch.float32).cpu()
    len_X = torch.tensor(length, dtype=torch.float32).cpu()
    with torch.no_grad():
        new_outputs = loaded_model(test_X, len_X).squeeze()

    new_outputs = torch.sigmoid(new_outputs)


    # y_pred_2 = [1 if prob >= 0.2 else 0 for prob in outputs]
    # y_pred_5 = [1 if prob >= 0.5 else 0 for prob in outputs]
    # y_pred_35 = [1 if prob >= 0.35 else 0 for prob in outputs]

    # for i, p in zip(train_id, y_pred):
    #     if p == 1:
    #         if i in output_dict:
    #             output_dict[i] += 1
    #         else:
    #             output_dict[i] = 1
                                                                                                                                                              
    # y_pred_ls_2.extend(y_pred_2)  
    # y_pred_ls_5.extend(y_pred_5)
    # y_pred_ls_35.extend(y_pred_35)
    # outputs_ls.extend(outputs)
    # labels_ls.extend(label)
    # id_ls.extend(train_id)
    # remaining_ls.extend([i]*len(label))
    new_outputs_ls.extend(new_outputs)
    labels_ls.extend(label)
    id_ls.extend(train_id)
    remaining_ls.extend([i]*len(label))


In [0]:
new_outputs_ls = [tensor.item() for tensor in new_outputs_ls]
t = pd.DataFrame(columns=['train_guest_id','remaining_days','label'])
t['label'] = labels_ls
t['train_guest_id'] = id_ls
t['remaining_days'] = remaining_ls
t['new_output'] = new_outputs_ls
t['rn'] = t.groupby('train_guest_id')['remaining_days'].transform('count')

t_df = spark.createDataFrame(t)

In [0]:
br = spark.read.parquet('dbfs:/mnt/proddatalake/dev/TierImminent/data/BR_test.parquet')
success_cus = br.groupby('train_guest_id').agg(F.sum('label').alias('success')).where('success > 0').select('train_guest_id')
analysis_df = br.join(t_df, on=['train_guest_id','remaining_days'], how='inner').join(success_cus, on='train_guest_id', how='leftanti')
above_thresh_cus = analysis_df.groupby('train_guest_id').agg(F.max('new_output').alias('percentage')).where('percentage > 0.5').select('train_guest_id')

In [0]:
target_analysis = analysis_df.\
    join(above_thresh_cus, on='train_guest_id', how='inner').join(customer_data, analysis_df.train_guest_id == customer_data.guest_id, how='inner').\
        join(cfa, analysis_df.train_guest_id == cfa.guest_id, how='inner').\
            select('train_guest_id',
                    'transactiondate',
                    'tiercredit',
                    'cuml_tc',
                    'trip_duration',
                    'TripLodgingStatus',
                    'gtc_percentage',
                    'ngtc_percentage',
                    'total_tc',
                    'target_tc',
                    'new_output',
                    'rn','playertypeltd_corporate', 'l18m_ATW')
target_analysis = target_analysis.join(trip_data,
                                       (target_analysis.train_guest_id == trip_data.guest_id) &(target_analysis.transactiondate >= trip_data.TripStart) & (target_analysis.transactiondate <= trip_data.TripEnd), how='inner')

In [0]:
target_analysis.display()

In [0]:
target_analysis.groupby('train_guest_id').agg(F.max('playertypeltd_corporate').alias('game')).groupby('game').count().display()

In [0]:
target_analysis.groupby('TripLodgingStatus').count().display()

In [0]:
re = 254
guest_id = 34617619
cus_list = df.filter(df.remaining_days == re).select('train_guest_id')
temp_df = df.join(cus_list, (df.train_guest_id == cus_list.train_guest_id) & (df.remaining_days >= re), how = 'inner').drop(df.train_guest_id)

temp_df = temp_df.withColumn('rn', F.row_number().over(Window.partitionBy('train_guest_id').orderBy('remaining_days'))).where('rn <= 6')

# train_np, label, length, train_id = create_sequences_and_labels(temp_df, features, 6)

# X = torch.tensor(train_np, dtype=torch.float32).cpu()
# len_X = torch.tensor(length, dtype=torch.float32).cpu()
# with torch.no_grad():
#     outputs = loaded_model(X, len_X).squeeze()
# outputs = torch.sigmoid(outputs)


features = ['tiercredit', 'remaining_days', 'days_num', 'cuml_tc', 'trip_duration','trip_day','TripLodgingStatus', 'highest_trip_tier', 'gtc_percentage', 'ngtc_percentage', 'total_tc', 'trip_num', 'trip_days', 'earliest_trip', 'lodger_percentage', 'local_percentage', 'target_tc']
temp_df.where(f'train_guest_id = {guest_id}').select('train_guest_id','transactiondate','change_assigned','label', *features, 'features_array').display()

In [0]:
test_X = torch.tensor(X, dtype=torch.float32).cpu()
len_X = torch.tensor(l, dtype=torch.float32).cpu()
with torch.no_grad():
    outputs = loaded_model(test_X, len_X).squeeze()
outputs = torch.sigmoid(outputs)

In [0]:
fpr, tpr, thresholds = roc_curve(y, outputs)
auc_score = roc_auc_score(y, outputs)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="best")
plt.grid()
for i in range(0, len(thresholds), len(thresholds)//20):
    plt.text(fpr[i], tpr[i], f'{thresholds[i]:.2f}', fontsize=8, color='red')
#plt.savefig("/Workspace/Users/609399@mgmresorts.com/Tier Imminent/model_performance_metric/sim_2024_roc.png")
plt.show()
plt.close() 

In [0]:
for i in range(0,len(thresholds)):
    if thresholds[i] < 0.5:
        print(i)
        break

In [0]:
fpr[9028]